### 1) Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

Предпочтительней использовать метрику precision@k, поскольку она дает нам точное представление о доле релевантных рекомендаций, в то время как hit_rate@k просто показывает наличие или отсутствие таковых.

<b> Примеры, где можно использовать hit_rate <br> </b>
Пример 1. Онлайн-кинотеатр <br>
Пользователь - зритель, товар - фильм. Когда пользователь заходит в сервис, нам нужно удержать его еще на один фильм, здесь и сейчас. <br>
Пример 2. Предложение однотипных дополнительных товаров к исходной покупке. <br>
Пользователь - покупка, товар - товар. Если человек покупает игровую мышку, то мы рекомендуем ему несколько игровых ковриков, и нас утроит что он купит только один. <br>
Пример 3. Любые магазины, ориентированные на подарочные товары (игрушки, ювелирка и т.д.) <br>
Пользователь - покупатель, товар - товар. Задача рекомендательной системы в том, чтобы помочь клиенту выбрать подарок и не отпустить его в более удобный магазин.

### 2)В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

Мы используем логарфим, потому что его функция возрастает более плавно и со временем скорость возрастания падает (в отличии от y=x). Таким образом, мы сдерживаем рост знаменателя. Можем использовать $$y= \sqrt x$$


### 3)Какие еще матрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована)

### 4) Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [4]:
import numpy as np
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

#### pr@8

In [5]:
def pr_at_k(boughted, recommended, k): #pr@k
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(boughted_list, recommended_list)
    
    pr = flags.sum() / len(recommended_list)
    
    return pr
    

In [6]:
pr_at_k(boughted, recommended, k=8)

0.5

#### rec@8

In [7]:
def rec_at_k(boughted, recommended, k): #rec@k
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(boughted_list, recommended_list)
    rec = flags.sum() / len(boughted_list)
    
    return rec

In [8]:
rec_at_k(boughted, recommended, k=8)

0.6666666666666666

#### AP@8

In [49]:
def ap_at_k(boughted, recommended, k): #ap@k
    boughted_list = boughted
    recommended_list = recommended
    flags = np.isin(recommended_list, boughted_list)
    
    if sum(flags) == 0:
        return 0
    
    k = sum(flags)
    sum_ = 0
    
    for i in range(1, k+1):
        if flags[i] == True:
            sum_ += pr_at_k(boughted_list, recommended_list, k=i)
    
    result = sum_ / sum(flags)
    
    return result
            

In [41]:
ap_at_k(boughted, recommended, k=8)

0.32

#### NDCG@8

In [58]:
def dcg(flags):
    result = flags[0]/1
    k = len(flags)
    
    for i in range(1, k):
        result += flags[i]/np.log(i+1)
    
    return result/k

In [64]:
def ndcg_a_k(boughted, recommended, k): 
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(recommended_list, boughted_list)
    len_ = len(recommended_list)
    
    dcg_ = dcg(flags)
    idcg = dcg(np.ones(len_))
    
    return dcg_ / idcg
    
    

In [71]:
ndcg_a_k(boughted, recommended, k=8)

0.5653142737255068

#### RR@8

In [108]:
def rec_r(boughted, rec_item):
    rank = 0
    for i, item_bought in enumerate(boughted):
        if rec_item == item_bought:
            return 1/(i+1)
            #rank = 1/(i+1)
    return rank

def mrr_at_k(boughted, recommended, k): 
    boughted_list = boughted
    recommended_list = recommended[:k]
    
    global_rank = 0
    for item in recommended_list:
        global_rank += rec_r(boughted_list, item)
    
    return global_rank / len(recommended_list)

In [112]:
mrr_at_k(boughted, recommended, k=8)

0.11875